In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM, Dense, Dropout, GRU
from keras.callbacks import EarlyStopping

In [ ]:
#read in data
df = pd.read_csv('data/hourly_nm.csv',index_col='Date/Time')
df.head()

In [ ]:
print(df.info())
print()
print(df.describe())

In [ ]:
#split data into training and testing, testing data will be one month
start_test = '2018-11-31'

train, test = df.loc[:start_test], df.loc[start_test:]

In [ ]:
train.tail(1)

In [ ]:
test.head(1)

In [ ]:
print(len(train))
print(len(test))

In [ ]:
# scale the data using MinMax Scaler from -1 to 1 as LSTM & GRU has a default tanh activation function
SCALER = MinMaxScaler(feature_range=(-1,1))

scaler = SCALER.fit(train.to_numpy())

train_scaled = scaler.transform(train.to_numpy())
test_scaled = scaler.transform(test.to_numpy())

In [ ]:
# create a function to split the datasets into two week windows
timestep = 24*7*2 # 24hours,7days,2weeks

def create_dataset(dataset, timestep=timestep):
    """
    Function which creates two week chunks of x_train data, and a single
    value for y_train.
    """
    X, y = [], []
    for i in range(len(dataset)):
        target_value = i + timestep
        if target_value == len(dataset):
            break
        feature_chunk, target = dataset[i:target_value, 1:], dataset[target_value, 0]
        X.append(feature_chunk)
        y.append(target)

    return np.array(X), np.array(y)

In [ ]:
#create x_train, y_train, X_test,y_test
X_train, y_train = create_dataset(train_scaled)
X_test, y_test = create_dataset(test_scaled)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# use sample of th data to train network to have a rough understanding of hyperparameters
samp_len = int(len(X_train)*0.5)

X_sample_train, y_sample_train = X_train[:samp_len], y_train[:samp_len]

In [ ]:
print(X_sample_train.shape)
print(y_sample_train.shape)

In [ ]:
# create X_train, y_train, X_test, y_test datasets
# create a function to build a stacked GRU model
# input needs to be [samples, timesteps, features]
def create_model(X_train, y_train):
    units = 32
    dropout = 0.05
    epochs = 35
    batch_size = 14
    optimizer = keras.optimizers.Adam(learning_rate=0.0005)
    early_stopping = EarlyStopping(patience=7, monitor='loss')

    model = keras.Sequential()

    #model.add(LSTM(units=units, dropout=dropout, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

    #model.add(LSTM(units=units, dropout=dropout))

    model.add(GRU(units=units))

    model.add(Dense(units=1))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    history = model.fit(X_train, y_train, validation_split=0.3, shuffle=False,
              epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])

    return model, history

In [ ]:
# function to predict a single value
def single_prediction(model, history, timestep=timestep):

        history = np.array(history)
        history = history.reshape(history.shape[0]*history.shape[1], history.shape[2])

        input_value = history[-timestep:]
        input_value = input_value.reshape(1, input_value.shape[0], input_value.shape[1])

        yhat = model.predict(input_value, verbose=0)
        return yhat

In [ ]:
# function which takes first test chunk, makes a prediction, add the test chunk back into training data
#to make next prediction

def walk_forward_prediction(X_train, y_train, X_test, timestep):

    MODEL, history = create_model(X_train=X_train, y_train=y_train)
    hist_train = [i for i in X_train]
    predictions = []

    for i in range(len(X_test)):
        test = X_test[i]
        yhat = single_prediction(model=MODEL, history=hist_train, timestep=timestep)
        predictions.append(yhat)
        hist_train.append(test)

    return predictions, history, MODEL

In [ ]:
def prior_inverse(features, targets):
    '''
    Append prediction value to test dataset and return a test shape format.
    '''
    dataset = []

    for i in range(features.shape[0]):
        last_row, target = features[i][0], targets[i]
        appended = np.append(last_row, target)
        dataset.append(appended)

    return np.array(dataset)

In [ ]:
#run experiemnt returning the real, predicted values
def experiment(X_train, y_train, X_test, timestep):

    pred_seq, history, MODEL = walk_forward_prediction(X_train, y_train, X_test, timestep)

    pred_seq = np.array(pred_seq).reshape(-1)

    pred = prior_inverse(X_test, pred_seq)
    real = prior_inverse(X_test, y_test)

    inv_pred = scaler.inverse_transform(pred)
    inv_real = scaler.inverse_transform(real)

    power_pred = inv_pred[:,-1]
    power_real = inv_real[:,-1]

    return power_real, power_pred, history, MODEL

In [ ]:
power_real, power_pred, history, MODEL = experiment(X_train, y_train, X_test, timestep)

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
#plot validation and training convergence graph
plt.figure(figsize=(10,5))
plt.plot(loss, label='train')
plt.plot(val_loss, label='validation')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('MSE')
plt.title('GRU Training Validation Loss')
plt.tight_layout()
plt.savefig('figures/GRU_train_val_plot.png')
plt.show()

In [ ]:
x_plot = test[timestep:].index
pred_df = pd.DataFrame({'Date':x_plot, 'Prediction': power_pred, 'True': power_real})
pred_df.set_index('Date', inplace=True)

In [ ]:
pred_df2 = pred_df['2018-12-15 01:00:00	':'2018-12-29 02:00:00 ']

In [ ]:
#plot predictions
pred_df2.plot(figsize=(10,5))
plt.title('Predicted Power vs Actual Power with GRU an Model.')
plt.ylabel('Power(KWh)')
plt.tight_layout()
plt.savefig('figures/GRU_prediction.png')
plt.show()

In [ ]:
#compute metrics
rmse = np.sqrt(mean_squared_error(pred_df2['True'], pred_df2['Prediction']))
mae = mean_absolute_error(pred_df2['True'], pred_df2['Prediction'])
r2 = r2_score(pred_df2['True'], pred_df2['Prediction'])
print('RMSE: {}\nMAE: {}\nR2: {}'.format(round(rmse,2),round(mae,2), round(r2,2)))